In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import interact, fixed

from darts import TimeSeries
from darts.models import NBEATSModel

plt.rcParams["figure.figsize"] = (14, 4)
pd.options.display.max_columns = 500

In [4]:
daily_train = pd.read_csv("../data/processed/daily-train-base.csv", parse_dates=["ds"])
daily_train

,id,ds,t,y,category
0,1,1994-03-01 12:00:00,1,1017.1,Macro
1,1,1994-03-02 12:00:00,2,1019.3,Macro
2,1,1994-03-03 12:00:00,3,1017.0,Macro
3,1,1994-03-04 12:00:00,4,1019.2,Macro
4,1,1994-03-05 12:00:00,5,1018.7,Macro
...,...,...,...,...,...
9964653,4227,2012-04-22 12:00:00,4436,15111.5,Other
9964654,4227,2012-04-23 12:00:00,4437,15154.0,Other
9964655,4227,2012-04-24 12:00:00,4438,15208.6,Other
9964656,4227,2012-04-25 12:00:00,4439,15108.0,Other


In [11]:
tss = []
for key in daily_train.id.unique():
    ts = TimeSeries.from_dataframe(
        daily_train.loc[daily_train.id == key, ["ds", "y"]],
        time_col="ds",
        value_cols="y",
    )
    tss.append(ts)

In [25]:
model_nbeats = NBEATSModel(
    input_chunk_length=28, 
    output_chunk_length=14, 
    generic_architecture=False, 
    num_stacks=30, 
    num_blocks=1, 
    num_layers=4, 
    layer_widths=256, 
    expansion_coefficient_dim=5, 
    trend_polynomial_degree=2, 
)

In [26]:
model_nbeats.fit(tss)

[2022-06-23 19:56:58,704] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 41754306 samples.
[2022-06-23 19:56:58,704] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 41754306 samples.
2022-06-23 19:56:58 darts.models.forecasting.torch_forecasting_model INFO: Train dataset contains 41754306 samples.
[2022-06-23 19:56:58,715] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
[2022-06-23 19:56:58,715] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
2022-06-23 19:56:58 darts.models.forecasting.torch_forecasting_model INFO: Time series values are 64-bits; casting model to float64.
2022-06-23 19:56:58 pytorch_lightning.utilities.distributed INFO: GPU available: False, used: False
2022-06-23 19:56:58 pytorch_lightning.utilities.distributed INFO: TPU available: False, using: 0 TPU cores
2022-06-23

Training: 0it [00:00, ?it/s]

C:\Users\ariel\anaconda3\envs\m4-forecasting\lib\site-packages\pytorch_lightning\trainer\trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
model_nbeats.save_model("../data/models/model_nbeats.pth.tar")